# Importing Libraries

In [3]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [5]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.models import Model

In [7]:
device = tf.device("GPU:0" if tf.config.experimental.list_physical_devices('GPU') else "CPU")

# Model

In [10]:
base_model = VGG19(weights='imagenet', include_top=False)
base_model.trainable = False

layer_names = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block4_conv2', 'block5_conv1']
outputs = [base_model.get_layer(name).output for name in layer_names]

model = Model(inputs=base_model.input, outputs=outputs)


80134624/80134624 ━━━━━━━━━━━━━━━━━━━━ 47s 1us/step


In [12]:
def model_activations(input_img, model):
    activations = model(input_img)
    features = {layer_names[i]: activations[i] for i in range(len(layer_names))}
    return features

In [14]:
def preprocess_image(image_path):
    img = kimage.load_img(image_path, target_size=(300, 300))
    img = kimage.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((300, 300, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

In [16]:
content_image = preprocess_image("/Users/harshsingh/Desktop/Style transfer/Dataset/monaLisa.jpg")
style_image = preprocess_image("/Users/harshsingh/Desktop/Style transfer/Dataset/MonaLisaStyle.jpg")

print("Content shape:", content_image.shape)

Content shape: (1, 300, 300, 3)


In [18]:
def gram_matrix(img_features):
    result = tf.linalg.einsum('bijc,bijd->bcd', img_features, img_features)
    input_shape = tf.shape(img_features)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

In [20]:
style_weights = {"block1_conv1": 1.0, 
                 "block2_conv1": 0.8,
                 "block3_conv1": 0.4,
                 "block4_conv1": 0.2,
                 "block5_conv1": 0.1}

content_weight = 100
style_weight = 1e8

optimizer = tf.optimizers.Adam(learning_rate=0.007)

In [22]:
content_features = model_activations(content_image, model)
style_features = model_activations(style_image, model)

style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

target_image = tf.Variable(content_image, dtype=tf.float32)

@tf.function
def compute_loss(target):
    target_features = model_activations(target, model)
    content_loss = tf.reduce_mean((content_features['block4_conv2'] - target_features['block4_conv2']) ** 2)
    
    style_loss = 0
    for layer in style_weights:
        style_gram = style_grams[layer]
        target_gram = gram_matrix(target_features[layer])
        _, d, h, w = target_gram.shape
        style_loss += style_weights[layer] * tf.reduce_mean((target_gram - style_gram) ** 2) / (d * h * w)
    
    total_loss = content_weight * content_loss + style_weight * style_loss
    return total_loss

@tf.function
def train_step(target):
    with tf.GradientTape() as tape:
        loss = compute_loss(target)
    grad = tape.gradient(loss, target)
    optimizer.apply_gradients([(grad, target)])
    return loss

epochs = 4000
print_after = 500

for i in range(1, epochs + 1):
    loss = train_step(target_image)
    
    if i % 10 == 0:
        print(f"Epoch {i}, Loss: {loss.numpy()}")
    
    if i % print_after == 0:
        img = deprocess_image(target_image.numpy())
        plt.imshow(img)
        plt.title(f"Epoch {i}")
        plt.show()
        plt.imsave(f'{i}.png', img)


ValueError: in user code:

    File "/var/folders/4k/g_jvzmx51w5cq6jkh2ybcynh0000gn/T/ipykernel_2021/1067025086.py", line 26, in train_step  *
        loss = compute_loss(target)
    File "/var/folders/4k/g_jvzmx51w5cq6jkh2ybcynh0000gn/T/ipykernel_2021/1067025086.py", line 17, in compute_loss  *
        _, d, h, w = target_gram.shape

    ValueError: not enough values to unpack (expected 4, got 3)


In [24]:
import tensorflow as tf
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image as kimage
from tensorflow.keras.models import Model
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

device = tf.device("GPU:0" if tf.config.experimental.list_physical_devices('GPU') else "CPU")

def preprocess_image(image_path):
    img = kimage.load_img(image_path, target_size=(300, 300))
    img = kimage.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.vgg19.preprocess_input(img)
    return img

def deprocess_image(img):
    img = img.reshape((300, 300, 3))
    img[:, :, 0] += 103.939
    img[:, :, 1] += 116.779
    img[:, :, 2] += 123.68
    img = img[:, :, ::-1]
    img = np.clip(img, 0, 255).astype('uint8')
    return img

def save_image(img, filename):
    img = deprocess_image(img)
    plt.imsave(filename, img)

base_model = VGG19(weights='imagenet', include_top=False)
base_model.trainable = False

layer_names = ['block1_conv1', 'block2_conv1', 'block3_conv1', 'block4_conv1', 'block4_conv2', 'block5_conv1']
outputs = [base_model.get_layer(name).output for name in layer_names]

model = Model(inputs=base_model.input, outputs=outputs)

def model_activations(input_img, model):
    activations = model(input_img)
    features = {layer_names[i]: activations[i] for i in range(len(layer_names))}
    return features

def gram_matrix(img_features):
    result = tf.linalg.einsum('bijc,bijd->bcd', img_features, img_features)
    input_shape = tf.shape(img_features)
    num_locations = tf.cast(input_shape[1] * input_shape[2], tf.float32)
    return result / num_locations

content_image = preprocess_image("/Users/harshsingh/Desktop/Style transfer/Dataset/monaLisa.jpg")
style_image = preprocess_image("/Users/harshsingh/Desktop/Style transfer/Dataset/MonaLisaStyle.jpg")

print("Content shape:", content_image.shape)

style_weights = {"block1_conv1": 1.0, 
                 "block2_conv1": 0.8,
                 "block3_conv1": 0.4,
                 "block4_conv1": 0.2,
                 "block5_conv1": 0.1}

content_weight = 100
style_weight = 1e8

optimizer = tf.optimizers.Adam(learning_rate=0.007)

content_features = model_activations(content_image, model)
style_features = model_activations(style_image, model)

style_grams = {layer: gram_matrix(style_features[layer]) for layer in style_features}

target_image = tf.Variable(content_image, dtype=tf.float32)

@tf.function
def compute_loss(target):
    target_features = model_activations(target, model)
    content_loss = tf.reduce_mean((content_features['block4_conv2'] - target_features['block4_conv2']) ** 2)
    
    style_loss = 0
    for layer in style_weights:
        style_gram = style_grams[layer]
        target_gram = gram_matrix(target_features[layer])
        target_shape = tf.shape(target_gram)
        _, d, h, w = target_shape[0], target_shape[1], target_shape[2], target_shape[3]
        style_loss += style_weights[layer] * tf.reduce_mean((target_gram - style_gram) ** 2) / (d * h * w)
    
    total_loss = content_weight * content_loss + style_weight * style_loss
    return total_loss

@tf.function
def train_step(target):
    with tf.GradientTape() as tape:
        loss = compute_loss(target)
    grad = tape.gradient(loss, target)
    optimizer.apply_gradients([(grad, target)])
    return loss

epochs = 4000
print_after = 500

for i in range(1, epochs + 1):
    loss = train_step(target_image)
    
    if i % 10 == 0:
        print(f"Epoch {i}, Loss: {loss.numpy()}")
    
    if i % print_after == 0:
        img = target_image.numpy()
        save_image(img, f'{i}.png')
        plt.imshow(deprocess_image(img))
        plt.title(f"Epoch {i}")
        plt.show()


Content shape: (1, 300, 300, 3)


ValueError: in user code:

    File "/var/folders/4k/g_jvzmx51w5cq6jkh2ybcynh0000gn/T/ipykernel_2021/2443125723.py", line 92, in train_step  *
        loss = compute_loss(target)
    File "/var/folders/4k/g_jvzmx51w5cq6jkh2ybcynh0000gn/T/ipykernel_2021/2443125723.py", line 83, in compute_loss  *
        _, d, h, w = target_shape[0], target_shape[1], target_shape[2], target_shape[3]

    ValueError: slice index 3 of dimension 0 out of bounds. for '{{node strided_slice_5}} = StridedSlice[Index=DT_INT32, T=DT_INT32, begin_mask=0, ellipsis_mask=0, end_mask=0, new_axis_mask=0, shrink_axis_mask=1](Shape_1, strided_slice_5/stack, strided_slice_5/stack_1, strided_slice_5/stack_2)' with input shapes: [3], [1], [1], [1] and with computed input tensors: input[1] = <3>, input[2] = <4>, input[3] = <1>.


In [20]:
from __future__ import division
from torchvision import models
from torchvision import transforms
from PIL import Image
import argparse
import torch
import torchvision
import torch.nn as nn
import numpy as np
import sys

In [10]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_image(image_path, transform=None, max_size=None, shape=None):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)
    
    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.ANTIALIAS)
    
    if shape:
        image = image.resize(shape, Image.LANCZOS)
    
    if transform:
        image = transform(image).unsqueeze(0)
    
    return image.to(device)

In [12]:
class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28'] 
        self.vgg = models.vgg19(pretrained=True).features
        
    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features

In [14]:
def main(config):
    
    # Image preprocessing
    # VGGNet was trained on ImageNet where images are normalized by mean=[0.485, 0.456, 0.406] and std=[0.229, 0.224, 0.225].
    # We use the same normalization statistics here.
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                             std=(0.229, 0.224, 0.225))])
    
    # Load content and style images
    # Make the style image same size as the content image
    content = load_image('/Users/harshsingh/Desktop/Style transfer/Dataset/monaLisa.jpg', transform, max_size=config.max_size)
    style = load_image('/Users/harshsingh/Desktop/Style transfer/Dataset/MonaLisaStyle.jpg', transform, shape=[content.size(2), content.size(3)])
    
    # Initialize a target image with the content image
    target = content.clone().requires_grad_(True)
    
    optimizer = torch.optim.Adam([target], lr=config.lr, betas=[0.5, 0.999])
    vgg = VGGNet().to(device).eval()
    
    for step in range(config.total_step):
        
        # Extract multiple(5) conv feature vectors
        target_features = vgg(target)
        content_features = vgg(content)
        style_features = vgg(style)

        style_loss = 0
        content_loss = 0
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            # Compute content loss with target and content images
            content_loss += torch.mean((f1 - f2)**2)

            # Reshape convolutional feature maps
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)

            # Compute gram matrix
            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())

            # Compute style loss with target and style images
            style_loss += torch.mean((f1 - f3)**2) / (c * h * w) 
        
        # Compute total loss, backprop and optimize
        loss = content_loss + config.style_weight * style_loss 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % config.log_step == 0:
            print ('Step [{}/{}], Content Loss: {:.4f}, Style Loss: {:.4f}' 
                   .format(step+1, config.total_step, content_loss.item(), style_loss.item()))

        if (step+1) % config.sample_step == 0:
            # Save the generated image
            denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().squeeze()
            img = denorm(img).clamp_(0, 1)
            torchvision.utils.save_image(img, 'output-{}.png'.format(step+1))


In [ ]:
config = {
    'content_path': '/path/to/content/image.jpg',
    'style_path': '/path/to/style/image.jpg',
    'output_dir': './output',
    'max_size': 400,
    'total_step': 2000,
    'log_step': 10,
    'sample_step': 500,
    'style_weight': 100,
    'lr': 0.003
}

# Create output directory if not exists
if not os.path.exists(config['output_dir']):
    os.makedirs(config['output_dir'])

# Run the main function
main(config)

In [22]:

if __name__ == "__main__":
    parser = argparse.ArgumentParser()
    parser.add_argument('--content', type=str, default='png/content.png')
    parser.add_argument('--style', type=str, default='png/style.png')
    parser.add_argument('--max_size', type=int, default=400)
    parser.add_argument('--total_step', type=int, default=2000)
    parser.add_argument('--log_step', type=int, default=10)
    parser.add_argument('--sample_step', type=int, default=500)
    parser.add_argument('--style_weight', type=float, default=100)
    parser.add_argument('--lr', type=float, default=0.003)
    config = parser.parse_args()
    print(config)
    main(config)

AttributeError: module 'PIL.Image' has no attribute 'ANTIALIAS'

In [ ]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_image(image_path, transform=None, max_size=None, shape=None):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)
    
    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.LANCZOS)
    
    if shape:
        image = image.resize(shape, Image.LANCZOS)
    
    if transform:
        image = transform(image).unsqueeze(0)
    
    return image.to(device)
class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28'] 
        self.vgg = models.vgg19(pretrained=True).features
        
    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features

def main(config):
    
    # Image preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                             std=(0.229, 0.224, 0.225))])
    
    # Load content and style images
    # Make the style image the same size as the content image
    content = load_image(config['content_path'], transform, max_size=config['max_size'])
    style = load_image(config['style_path'], transform, shape=[content.size(2), content.size(3)])
    
    # Initialize a target image with the content image
    target = content.clone().requires_grad_(True)
    
    optimizer = torch.optim.Adam([target], lr=config['lr'], betas=[0.5, 0.999])
    vgg = VGGNet().to(device).eval()
    
    for step in range(config['total_step']):
        
        # Extract multiple(5) conv feature vectors
        target_features = vgg(target)
        content_features = vgg(content)
        style_features = vgg(style)

        style_loss = 0
        content_loss = 0
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            # Compute content loss with target and content images
            content_loss += torch.mean((f1 - f2)**2)

            # Reshape convolutional feature maps
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)

            # Compute gram matrix
            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())

            # Compute style loss with target and style images
            style_loss += torch.mean((f1 - f3)**2) / (c * h * w) 
        
        # Compute total loss, backprop and optimize
        loss = content_loss + config['style_weight'] * style_loss 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % config['log_step'] == 0:
            print ('Step [{}/{}], Content Loss: {:.4f}, Style Loss: {:.4f}' 
                   .format(step+1, config['total_step'], content_loss.item(), style_loss.item()))

        if (step+1) % config['sample_step'] == 0:
            # Save the generated image
            denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().squeeze()
            img = denorm(img).clamp_(0, 1)
            torchvision.utils.save_image(img, os.path.join(config['output_dir'], 'output-{}.png'.format(step+1)))

# Configuration
config = {
    'content_path': '/Users/harshsingh/Desktop/Style transfer/Dataset/content_1.jpg',
    'style_path': '/Users/harshsingh/Desktop/Style transfer/Dataset/MonaLisaStyle.jpg',
    'output_dir': '/Users/harshsingh/Desktop/Style transfer',
    'max_size': 400, 
    'total_step': 2000,
    'log_step': 10,
    'sample_step': 500,
    'style_weight': 100,
    'lr': 0.003
}

# Create output directory if not exists
if not os.path.exists(config['output_dir']):
    os.makedirs(config['output_dir'])

# Run the main function
main(config)


Step [10/2000], Content Loss: 3.7166, Style Loss: 508.5862
Step [20/2000], Content Loss: 8.8290, Style Loss: 348.2332
Step [30/2000], Content Loss: 13.0189, Style Loss: 262.6015
Step [40/2000], Content Loss: 16.4286, Style Loss: 209.0936
Step [50/2000], Content Loss: 19.1647, Style Loss: 172.7243
Step [60/2000], Content Loss: 21.4725, Style Loss: 146.3952
Step [70/2000], Content Loss: 23.4462, Style Loss: 126.6476
Step [80/2000], Content Loss: 25.1690, Style Loss: 111.4013
Step [90/2000], Content Loss: 26.6881, Style Loss: 99.3812
Step [100/2000], Content Loss: 28.0319, Style Loss: 89.7030
Step [110/2000], Content Loss: 29.2331, Style Loss: 81.7800
Step [120/2000], Content Loss: 30.3219, Style Loss: 75.1866
Step [130/2000], Content Loss: 31.2947, Style Loss: 69.6356
Step [140/2000], Content Loss: 32.1707, Style Loss: 64.8970
Step [150/2000], Content Loss: 32.9644, Style Loss: 60.8053
Step [160/2000], Content Loss: 33.6949, Style Loss: 57.2299
Step [170/2000], Content Loss: 34.3635, Sty

In [6]:
import torch
import torch.nn as nn
import torchvision
from torchvision import models, transforms
from PIL import Image
import numpy as np
import os

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def load_image(image_path, transform=None, max_size=None, shape=None):
    """Load an image and convert it to a torch tensor."""
    image = Image.open(image_path)
    
    if max_size:
        scale = max_size / max(image.size)
        size = np.array(image.size) * scale
        image = image.resize(size.astype(int), Image.LANCZOS)
    
    if shape:
        image = image.resize(shape, Image.LANCZOS)
    
    if transform:
        image = transform(image).unsqueeze(0)
    
    return image.to(device)
class VGGNet(nn.Module):
    def __init__(self):
        """Select conv1_1 ~ conv5_1 activation maps."""
        super(VGGNet, self).__init__()
        self.select = ['0', '5', '10', '19', '28'] 
        self.vgg = models.vgg19(pretrained=True).features
        
    def forward(self, x):
        """Extract multiple convolutional feature maps."""
        features = []
        for name, layer in self.vgg._modules.items():
            x = layer(x)
            if name in self.select:
                features.append(x)
        return features

def main(config):
    # Image preprocessing
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.485, 0.456, 0.406), 
                             std=(0.229, 0.224, 0.225))])
    
    # Load content and style images
    # Make the style image the same size as the content image
    content = load_image(config['content_path'], transform, max_size=config['max_size'])
    style = load_image(config['style_path'], transform, shape=[content.size(2), content.size(3)])
    
    # Initialize a target image with the content image
    target = content.clone().requires_grad_(True)
    
    optimizer = torch.optim.Adam([target], lr=config['lr'], betas=[0.5, 0.999])
    vgg = VGGNet().to(device).eval()
    
    for step in range(config['total_step']):
        
        # Extract multiple(5) conv feature vectors
        target_features = vgg(target)
        content_features = vgg(content)
        style_features = vgg(style)

        style_loss = 0
        content_loss = 0
        for f1, f2, f3 in zip(target_features, content_features, style_features):
            # Compute content loss with target and content images
            content_loss += torch.mean((f1 - f2)**2)

            # Reshape convolutional feature maps
            _, c, h, w = f1.size()
            f1 = f1.view(c, h * w)
            f3 = f3.view(c, h * w)

            # Compute gram matrix
            f1 = torch.mm(f1, f1.t())
            f3 = torch.mm(f3, f3.t())

            # Compute style loss with target and style images
            style_loss += torch.mean((f1 - f3)**2) / (c * h * w) 
        
        # Compute total loss, backprop and optimize
            #   loss = content_loss + config['style_weight'] * style_loss 

        loss = config['alpha'] * content_loss + config['beta'] * style_loss 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (step+1) % config['log_step'] == 0:
            print ('Step [{}/{}], Content Loss: {:.4f}, Style Loss: {:.4f}' 
                   .format(step+1, config['total_step'], content_loss.item(), style_loss.item()))

        if (step+1) % config['sample_step'] == 0:
            # Save the generated image
            denorm = transforms.Normalize((-2.12, -2.04, -1.80), (4.37, 4.46, 4.44))
            img = target.clone().squeeze()
            img = denorm(img).clamp_(0, 1)
            torchvision.utils.save_image(img, os.path.join(config['output_dir'], 'output-{}.png'.format(step+1)))

# Configuration
config = {
    'content_path': '/Users/harshsingh/Desktop/Style transfer/Dataset/monaLisa.jpg',
    'style_path': '/Users/harshsingh/Desktop/Style transfer/Dataset/heart_style.png',
    'output_dir': '/Users/harshsingh/Desktop/Style transfer/mona_dil',
    'max_size': 360, 
    'total_step': 5000,
    'log_step': 10,
    'sample_step': 250,
    'style_weight': 100,
    'alpha': 1,  # Content loss weight
    'beta': 100, # Style loss weight
    'lr': 0.001
}
# Create output directory if not exists
if not os.path.exists(config['output_dir']):
    os.makedirs(config['output_dir'])

# Run the main function
main(config)


Step [10/5000], Content Loss: 1.2266, Style Loss: 7792.7632
Step [20/5000], Content Loss: 5.9670, Style Loss: 7498.9302
Step [30/5000], Content Loss: 13.1665, Style Loss: 7065.3926
Step [40/5000], Content Loss: 20.7102, Style Loss: 6575.8906
Step [50/5000], Content Loss: 27.0949, Style Loss: 6109.1709
Step [60/5000], Content Loss: 31.9386, Style Loss: 5683.2451
Step [70/5000], Content Loss: 35.6233, Style Loss: 5302.2534
Step [80/5000], Content Loss: 38.5533, Style Loss: 4963.3115
Step [90/5000], Content Loss: 40.9937, Style Loss: 4664.7139
Step [100/5000], Content Loss: 43.1280, Style Loss: 4402.3823
Step [110/5000], Content Loss: 45.0502, Style Loss: 4171.3662
Step [120/5000], Content Loss: 46.8084, Style Loss: 3967.0830
Step [130/5000], Content Loss: 48.4225, Style Loss: 3784.6677
Step [140/5000], Content Loss: 49.9230, Style Loss: 3620.0852
Step [150/5000], Content Loss: 51.3250, Style Loss: 3470.2720
Step [160/5000], Content Loss: 52.6428, Style Loss: 3332.2625
Step [170/5000], Co

KeyboardInterrupt: 